<a href="https://colab.research.google.com/github/PawelG-WWA/learning-features/blob/basic-data-analysis/PROW_pd4484.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About

The project's goal is to analyze dataset of factors impacting student performance. Dataset comes from [kaggle.com](https://www.kaggle.com/datasets/lainguyn123/student-performance-factors) and provides comprehensive overview of various factors affecting student final exam score. After analysis, I will introduce some Machine Learning models to classify and clasterize students to predict their potential so that teachers would know on whom they should focus on more.


# Loading basic libraries

Let's load some basic libraries for the analysis part

In [119]:
import pandas as pd
import numpy as np
from google.colab import userdata

# 1. Data research

Now we need to load and investigate the data to answer some quedtions:
- How the data looks like?
- Are there any missing values?
  - If some values are missing, how we should fill them? Should we drop observations with missing values?
- What are they types of values?
- Can we change type of values to something more meaningful/reasonable?
- What the dataset represents?

In [120]:
# load data into data frame
filepath = userdata.get('studentPerformanceFilePath')

df = pd.read_csv(filepath)
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [121]:
# show shape of the data frame: format (rows, columns)
df.shape

(6607, 20)

In [122]:
# show count of non null values in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6529 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

## 1.1 Data engineering

In this section my focus is to examine the data thoroughly. I will endeavor to understand the meaning and possible implications of variables and how it all relates to the real world context. This will provide me with a comprehensive understanding of the problem domain.

Let's start with investigating possible categorical and boolean variables.

In [123]:
# summary of value counts in potentially categorical/boolean variables
(df['Parental_Involvement'].value_counts(),
 df['Access_to_Resources'].value_counts(),
 df['Extracurricular_Activities'].value_counts(),
 df['Motivation_Level'].value_counts(),
 df['Internet_Access'].value_counts(),
 df['Family_Income'].value_counts(),
 df['Teacher_Quality'].value_counts(),
 df['School_Type'].value_counts(),
 df['Peer_Influence'].value_counts(),
 df['Learning_Disabilities'].value_counts(),
 df['Parental_Education_Level'].value_counts(),
 df['Distance_from_Home'].value_counts(),
 df['Gender'].value_counts())

(Parental_Involvement
 Medium    3362
 High      1908
 Low       1337
 Name: count, dtype: int64,
 Access_to_Resources
 Medium    3319
 High      1975
 Low       1313
 Name: count, dtype: int64,
 Extracurricular_Activities
 Yes    3938
 No     2669
 Name: count, dtype: int64,
 Motivation_Level
 Medium    3351
 Low       1937
 High      1319
 Name: count, dtype: int64,
 Internet_Access
 Yes    6108
 No      499
 Name: count, dtype: int64,
 Family_Income
 Low       2672
 Medium    2666
 High      1269
 Name: count, dtype: int64,
 Teacher_Quality
 Medium    3925
 High      1947
 Low        657
 Name: count, dtype: int64,
 School_Type
 Public     4598
 Private    2009
 Name: count, dtype: int64,
 Peer_Influence
 Positive    2638
 Neutral     2592
 Negative    1377
 Name: count, dtype: int64,
 Learning_Disabilities
 No     5912
 Yes     695
 Name: count, dtype: int64,
 Parental_Education_Level
 High School     3223
 College         1989
 Postgraduate    1305
 Name: count, dtype: int64,
 Dis

In [124]:
# change all properties with 3 values into category

low_high_gradation = ['Low', 'Medium', 'High']
school_gradation = ['High School', 'College', 'Postgraduate']
distance_gradation = ['Near', 'Moderate', 'Far']
peer_influence_gradation = ['Negative', 'Neutral', 'Positive']

def change_type_to_category(column, gradation_type):
  df[column] = pd.Categorical(df[column], categories=gradation_type, ordered=True)

for data_column in df[['Parental_Involvement', 'Access_to_Resources', 'Motivation_Level', 'Family_Income', 'Teacher_Quality']]:
  change_type_to_category(data_column, low_high_gradation)

change_type_to_category('Parental_Education_Level', school_gradation)
change_type_to_category('Distance_from_Home', distance_gradation)
change_type_to_category('Peer_Influence', peer_influence_gradation)


In [125]:
# Add boolean properties for those column which only have 2 values within
df['Extracurricular_Activities_boolean'] = df['Extracurricular_Activities'].map({'Yes': True, 'No': False})
df['Internet_Access_boolean'] = df['Internet_Access'].map({'Yes': True, 'No': False})
df['School_Type_IsPublic'] = df['School_Type'].map({'Public': True, 'Private': False})
df['Learning_Disabilities_boolean'] = df['Learning_Disabilities'].map({'Yes': True, 'No': False})
df['IsFemale'] = df['Gender'].map({'Female': True, 'Male': False})

In [126]:
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,...,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score,Extracurricular_Activities_boolean,Internet_Access_boolean,School_Type_IsPublic,Learning_Disabilities_boolean,IsFemale
0,23,84,Low,High,No,7,73,Low,Yes,0,...,No,High School,Near,Male,67,False,True,True,False,False
1,19,64,Low,Medium,No,8,59,Low,Yes,2,...,No,College,Moderate,Female,61,False,True,True,False,True
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,...,No,Postgraduate,Near,Male,74,True,True,True,False,False
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,...,No,High School,Moderate,Male,71,True,True,True,False,False
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,...,No,College,Near,Female,70,True,True,True,False,True


In [127]:
# Remove data with null values
#
# There is no sense in filling missing teacher quality, parental education level or distance from home
# Filling with anything would be filling with a random value not representing the reality at all, it owuld be a guess
#
# I decided to remove all rows with missing values as they constitute only ~3.5% of the whole dataset.
df.dropna(subset=['Teacher_Quality'], inplace=True)
df.dropna(subset=['Parental_Education_Level'], inplace=True)
df.dropna(subset=['Distance_from_Home'], inplace=True)

In [128]:
# let's see if scores are between 0 and 100. If the score is higher than maximum value
# or lower than minimum value, that's probably an error and the score should be rounded to maximum/minimum
scores_minmax = pd.DataFrame({
    'Previous_Score_min': min(df['Previous_Scores']),
    'Previous_Score_max': max(df['Previous_Scores']),
    'Exam_Score_min': min(df['Exam_Score']),
    'Exam_Score_max': max(df['Exam_Score'])
}, index=[0])

scores_minmax

,Previous_Score_min,Previous_Score_max,Exam_Score_min,Exam_Score_max
0,50,100,55,101


In [129]:
# we can see that Exam_Score_max = 101. We need to update all 101 to 100
df.loc[df['Exam_Score'] == 101, 'Exam_Score'] = 100
df['Exam_Score'].describe()

,Exam_Score
count,6378.000000
mean,67.251960
std,3.912884
min,55.000000
25%,65.000000
50%,67.000000
75%,69.000000
max,100.000000


In [130]:
# this is the final dataset we will work with:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6378 entries, 0 to 6606
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Hours_Studied                       6378 non-null   int64   
 1   Attendance                          6378 non-null   int64   
 2   Parental_Involvement                6378 non-null   category
 3   Access_to_Resources                 6378 non-null   category
 4   Extracurricular_Activities          6378 non-null   object  
 5   Sleep_Hours                         6378 non-null   int64   
 6   Previous_Scores                     6378 non-null   int64   
 7   Motivation_Level                    6378 non-null   category
 8   Internet_Access                     6378 non-null   object  
 9   Tutoring_Sessions                   6378 non-null   int64   
 10  Family_Income                       6378 non-null   category
 11  Teacher_Quality                    

## 1.2 Knowing data better

In this section we will focus more on data investigation. We will try to find dependencies between data and proportions between properties.

In general, our goal is to be able to improve exam results. To do that, we need to find out what impacts the exam score the most, what characterizes students who get the lowest or the highest exam score.

Let's find out some basic information and depct some dependecies with plots.

In [131]:
# Import plotly library for creating charts
import plotly.express as px
import plotly.graph_objects as go

In [132]:
# Let's remind ourselves how the dataset looks like
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,...,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score,Extracurricular_Activities_boolean,Internet_Access_boolean,School_Type_IsPublic,Learning_Disabilities_boolean,IsFemale
0,23,84,Low,High,No,7,73,Low,Yes,0,...,No,High School,Near,Male,67,False,True,True,False,False
1,19,64,Low,Medium,No,8,59,Low,Yes,2,...,No,College,Moderate,Female,61,False,True,True,False,True
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,...,No,Postgraduate,Near,Male,74,True,True,True,False,False
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,...,No,High School,Moderate,Male,71,True,True,True,False,False
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,...,No,College,Near,Female,70,True,True,True,False,True


In [133]:
# let's compare scores from previous and current exam.
#
# We want to take all the data except count, as we know what the count is already and count will look bad on a chart
# because of its huge value
scores_comparison = pd.DataFrame({
      'Previous_Scores': df['Previous_Scores'].describe(),
      'Exam_Score': df['Exam_Score'].describe()
    }).tail(-1)

scores_comparison

,Previous_Scores,Exam_Score
mean,75.066165,67.251960
std,14.400389,3.912884
min,50.000000,55.000000
25%,63.000000,65.000000
50%,75.000000,67.000000
75%,88.000000,69.000000
max,100.000000,100.000000


In [134]:
fig = go.Figure(
    data = [
        go.Bar(name='Previous_Scores', x=scores_comparison.index, y=scores_comparison['Previous_Scores']),
        go.Bar(name='Exam_Scores', x=scores_comparison.index, y=scores_comparison['Exam_Score'])
    ]
)

fig.update_layout(barmode='group', title='Comparison between Previous_Scores and current Exam_Scores', xaxis_title='property', yaxis_title='score')
fig.show()

# As we can see, current Exam_Scores in comparison to Previous_Score have:
# - lower mean and standard deviation
# - higher minimum score and first quartile
# - lower second and third quartiles

## 1.3 Finding score catalysts

Now we have some general idea about the dataset. We compared scores from previous and current exams. We don't know anything about the exams though - results are different but we don't know why - it might be for example that the current exam was harder than previous one or that the material was less understandable.

Nevertheless we can look for factors among students that impact the final result and that's we will focus on in this sub-section.

In [135]:
# First let's add some grades for score ranges:
# A: 90-100
# B: 75-89
# C: 65-74
# D: 50-64
# F: 0-49
#
# This setup will help us in grouping, and grouping will simplifiy the process of finding factors which impact score
def apply_grade(score):
  if score >= 93:
    return 'A+'
  elif score >= 87:
    return 'A'
  elif score >= 83:
    return 'A-'
  if score >= 78:
    return 'B+'
  elif score >= 74:
    return 'B'
  elif score >= 70:
    return 'B-'
  if score >= 67:
    return 'C+'
  elif score >= 63:
    return 'C'
  elif score >= 60:
    return 'C-'
  if score >= 55:
    return 'D+'
  elif score >= 50:
    return 'D'
  else:
    return 'F'
grade_categories = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'F']
df['Exam_grade'] = [apply_grade(score) for score in df['Exam_Score']]
df['Exam_grade'] = pd.Categorical(df['Exam_grade'], categories=grade_categories, ordered=True)
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,...,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score,Extracurricular_Activities_boolean,Internet_Access_boolean,School_Type_IsPublic,Learning_Disabilities_boolean,IsFemale,Exam_grade
0,23,84,Low,High,No,7,73,Low,Yes,0,...,High School,Near,Male,67,False,True,True,False,False,C+
1,19,64,Low,Medium,No,8,59,Low,Yes,2,...,College,Moderate,Female,61,False,True,True,False,True,C-
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,...,Postgraduate,Near,Male,74,True,True,True,False,False,B
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,...,High School,Moderate,Male,71,True,True,True,False,False,B-
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,...,College,Near,Female,70,True,True,True,False,True,B-


In [136]:
# Now, let's see what fraction of student get the given grades
counts = df['Exam_grade'].value_counts()
pd.DataFrame({
    'counts': counts,
    'percentage': round(counts/len(df), 5)
})

,counts,percentage
Exam_grade,,
C,2200,0.34494
C+,2029,0.31812
B-,1355,0.21245
C-,502,0.07871
B,171,0.02681
D+,66,0.01035
A+,19,0.00298
B+,16,0.00251
A,11,0.00172


In [137]:
grade_counts = df.groupby('Exam_grade', observed=False).count().iloc[:,0].rename('grade_counts')
grade_counts

,grade_counts
Exam_grade,
A+,19
A,11
A-,9
B+,16
B,171
B-,1355
C+,2029
C,2200
C-,502


### 1.3.1 Motivation level impact

As we can see below, Medium motivaiton level characterizes many students across grades. Low motivation level is higher among students with lower grades, while higher level is associated with people getting better grades.

This is a good suggestion to transform these values into weighted representatoins instead of simply assigning 1, 2 and 3 to them.

In [138]:
motivation_factor = df.groupby(['Exam_grade', 'Motivation_Level'], observed=False)['Exam_Score'].count()
motivation_factor = motivation_factor.reset_index(name='count')
motivation_factor = motivation_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
motivation_factor['%'] = round(motivation_factor['count']/motivation_factor['grade_counts'] * 100, 2)

fig = px.bar(
    motivation_factor,
    x='Exam_grade',
    y='%',
    color='Motivation_Level',
    barmode='group',
    title='Exam Grade and Motivation Level Distribution'
)

fig.show()

### 1.3.2 Parental involvement

Similarly to motivaiton level, parental involvement is higher among students with better grades, but is growing only to some extent (to B) and then it roughly stays the same.

We can clearly see though that low involvement grows among students with worse results.

In [139]:
parental_involvement_factor = df.groupby(['Exam_grade', 'Parental_Involvement'], observed=False)['Exam_Score'].count()
parental_involvement_factor = parental_involvement_factor.reset_index(name='count')
parental_involvement_factor = parental_involvement_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
parental_involvement_factor['%'] = round(parental_involvement_factor['count']/parental_involvement_factor['grade_counts'] * 100, 2)

fig = px.bar(
    parental_involvement_factor,
    x='Exam_grade',
    y='%',
    color='Parental_Involvement',
    barmode='group',
    title='Exam Grade and Parental Involvement Distribution'
)
fig.show()

### 1.3.3 Access to resources

Low availability to education resources characterizes students with worse results, while high availability is typical for people getting better results.

Medium is very neutral and will be dimmed while two other values will weight more.


In [140]:
access_to_resources_factor = df.groupby(['Exam_grade', 'Access_to_Resources'], observed=False)['Exam_Score'].count()
access_to_resources_factor = access_to_resources_factor.reset_index(name='count')
access_to_resources_factor = access_to_resources_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
access_to_resources_factor['%'] = round(access_to_resources_factor['count']/access_to_resources_factor['grade_counts'] * 100, 2)

fig = px.bar(
    access_to_resources_factor,
    x='Exam_grade',
    y='%',
    color='Access_to_Resources',
    barmode='group',
    title='Exam Grade and Access to Resources Distribution'
)
fig.show()

### 1.3.4 Distance from home

Looks like distance from home doesn't differ too much among students, it will be removed before creating the model as it doesn't seem to have an impact on grades received.

In [142]:
distance_from_home_factor = df.groupby(['Exam_grade', 'Distance_from_Home'], observed=False)['Exam_Score'].count()
distance_from_home_factor = distance_from_home_factor.reset_index(name='count')
distance_from_home_factor = distance_from_home_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
distance_from_home_factor['%'] = round(distance_from_home_factor['count']/distance_from_home_factor['grade_counts'] * 100, 2)

fig = px.bar(
    distance_from_home_factor,
    x='Exam_grade',
    y='%',
    color='Distance_from_Home',
    barmode='group',
    title='Exam Grade and Distance from Home'
)
fig.show()

### 1.3.5 Family income

From the chart it is clear that higher family income means better results.

In [144]:
family_income_factor = df.groupby(['Exam_grade', 'Family_Income'], observed=False)['Exam_Score'].count()
family_income_factor = family_income_factor.reset_index(name='count')
family_income_factor = family_income_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
family_income_factor['%'] = round(family_income_factor['count']/family_income_factor['grade_counts'] * 100, 2)
fig = px.bar(
    family_income_factor,
    x='Exam_grade',
    y='%',
    color='Family_Income',
    barmode='group',
    title='Exam Grade and Family Income Distribution'
)
fig.show()

### 1.3.6 Teacher quality

Looks like only high teacher qulity may have an impact on exam result. Medium and low quality teachers are represented almost equally.

In [146]:
teacher_quality_factor = df.groupby(['Exam_grade', 'Teacher_Quality'], observed=False)['Exam_Score'].count()
teacher_quality_factor = teacher_quality_factor.reset_index(name='count')
teacher_quality_factor = teacher_quality_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
teacher_quality_factor['%'] = round(teacher_quality_factor['count']/teacher_quality_factor['grade_counts'] * 100, 2)

fig = px.bar(
    teacher_quality_factor,
    x='Exam_grade',
    y='%',
    color='Teacher_Quality',
    barmode='group',
    title='Exam Grade and Teacher Quality Distribution'
)
fig.show()


### 1.3.7 Peer influence

Again, medium value is not trending in any direction, while we can clearly see that negative influence may be associated with worse grades and on the other hand, positive impact results in bette rgrades.

In [148]:
peer_influence_factor = df.groupby(['Exam_grade', 'Peer_Influence'], observed=False)['Exam_Score'].count()
peer_influence_factor = peer_influence_factor.reset_index(name='count')
peer_influence_factor = peer_influence_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
peer_influence_factor['%'] = round(peer_influence_factor['count']/peer_influence_factor['grade_counts'] * 100, 2)

fig = px.bar(
    peer_influence_factor,
    x='Exam_grade',
    y='%',
    color='Peer_Influence',
    barmode='group',
    title='Exam Grade and Peer Influence Distribution'
)
fig.show()

### 1.3.7 Parental level education

Students whose parents finished high school are more predistined to get a worse score in comparison to students whose parents graduated college.

It's just a raw data, there may be other factors involved, but it is some kind of information we can use in firther analysis, especially, because we can see trends in both directions (lower parent's education -> worse grades, better parent's education -> higher grades)

In [149]:
parental_education_level_factor = df.groupby(['Exam_grade', 'Parental_Education_Level'])['Exam_Score'].count()
parental_education_level_factor = parental_education_level_factor.reset_index(name='count')
parental_education_level_factor = parental_education_level_factor.merge(grade_counts, left_on='Exam_grade', right_index=True)
parental_education_level_factor['%'] = round(parental_education_level_factor['count']/parental_education_level_factor['grade_counts'] * 100, 2)

fig = px.bar(
    parental_education_level_factor,
    x='Exam_grade',
    y='%',
    color='Parental_Education_Level',
    barmode='group',
    title='Exam Grade and Parental Education Level Distribution'
)
fig.show()

<ipython-input-149-a3527ecdacfb>:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## 2. Searching for dependencies
